# ShadowKey Decryption Process

This notebook demonstrates how to decrypt files that were encrypted by ShadowKey. 

## Prerequisites
- You must have the `encryption_key.txt` file that was used during encryption
- The encrypted files should be in the same directory as this notebook
- Files should be prefixed with `e_` (e.g., `e_system.txt`)

## Process
1. Load the encryption key from `encryption_key.txt`
2. Identify encrypted files in the current directory
3. Decrypt each file to a timestamped output directory
4. Save the decrypted contents with original filenames

## Security Note
Keep your encryption key secure. If the key is lost, the encrypted files cannot be recovered.

In [ ]:
from cryptography.fernet import Fernet
import os
from datetime import datetime

def load_key():
    """Load the encryption key from file"""
    try:
        with open("encryption_key.txt", "rb") as key_file:
            return key_file.read()
    except FileNotFoundError:
        print("[-] Encryption key not found!")
        return None

def decrypt_file(encrypted_file, key, output_dir):
    """Decrypt a single file and save it to the output directory"""
    try:
        # Read encrypted data
        with open(encrypted_file, 'rb') as f:
            encrypted_data = f.read()
            
        # Decrypt the data
        fernet = Fernet(key)
        decrypted_data = fernet.decrypt(encrypted_data)
        
        # Create output filename (remove 'e_' prefix)
        output_name = encrypted_file[2:] if encrypted_file.startswith('e_') else encrypted_file
        output_path = os.path.join(output_dir, f"decrypted_{output_name}")
        
        # Write decrypted data
        with open(output_path, 'wb') as f:
            f.write(decrypted_data)
            
        print(f"[+] Successfully decrypted {encrypted_file} to {output_path}")
        return True
        
    except Exception as e:
        print(f"[-] Failed to decrypt {encrypted_file}: {str(e)}")
        return False

# Load the encryption key
key = load_key()
if not key:
    print("Cannot proceed without encryption key!")
else:
    # Create output directory with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = f"decrypted_files_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all encrypted files (prefixed with 'e_')
    encrypted_files = [f for f in os.listdir('.') if f.startswith('e_')]
    
    print(f"\nFound {len(encrypted_files)} encrypted files")
    print(f"Output directory: {output_dir}")
    print("\n=== Starting Decryption Process ===")
    
    # Decrypt each file
    successful = 0
    for enc_file in encrypted_files:
        if decrypt_file(enc_file, key, output_dir):
            successful += 1
            
    print(f"\nDecryption complete: {successful}/{len(encrypted_files)} files processed")